In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
%matplotlib qt5

In [2]:
import sys

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
from posarutils.other.read_data_and_build_rd import *

In [5]:
fmin = 5.725e9
fMAX = 5.875e9
f0 = 5.8e9
B = fMAX - fmin
T = 600e-6
alpha = 2 * (fMAX - fmin) / T
c = 3e8

In [99]:
t1 = np.arange( 0, T, 100e-9)
t2 = np.arange( -T/2, T/2, 100e-9)

In [113]:
rampsPerFile = 1500
samplesPerRamp = 6000
samplesPerUpRamp = 3000
nbFiles = 10

In [107]:
def sb1( t, r ):
    y = np.exp( -1j * 2  * np.pi * f0 * tau ) \
            * np.exp( 1j * np.pi * alpha * tau**2 ) \
            * np.exp(  -1j * 2 * np.pi * alpha * t * tau )
    return y

def sb2( t, r ):
    y = np.zeros( t.shape, dtype=complex )
    tau = 2 * r / c
    idx = np.where( t < tau )
    y[idx] = np.exp( -1j * 2  * np.pi * f0 * (T-tau) ) \
            * np.exp( 1j * np.pi * alpha * (T-tau)**2 ) \
            * np.exp(  -1j * 2 * np.pi * alpha * t[idx] * (T-tau) )
    idx = np.where( t > tau )
    y[idx] = np.exp( -1j * 2  * np.pi * f0 * tau ) \
            * np.exp( 1j * np.pi * alpha * tau**2 ) \
            * np.exp(  -1j * 2 * np.pi * alpha * (t[idx]-tau) * tau )
    return y

In [111]:
plt.figure()
r = 100
tau = 2 * r / c
plt.subplot( 211 )
plt.plot( np.real( sb1( t1[0:3000], r ) ), '.' )
plt.plot( np.real( sb2( t1[0:3000], r ) ) )
ax = plt.gca()
plt.subplot( 212, sharex=ax )
plt.plot( np.imag( sb1( t1[0:3000], r ) ) )
plt.plot( np.imag( sb2( t1[0:3000], r ) ) )
ax = plt.gca()
ax.set_xlim(0, 100)

(0, 100)

In [110]:
plt.figure()
r = 100
sb = np.real( sb2( t1[0:3000], r ) )
plt.subplot(311)
plt.plot( 20 * np.log10(np.abs(np.fft.ifft( sb ) ) ) )
plt.subplot(312)
plt.plot( np.real(np.fft.ifft( sb ) ) )
plt.subplot(313)
plt.plot( np.imag(np.fft.ifft( sb ) ) )

In [84]:
def sr( t, r ):
    
    tau = 2 * r / c
    
    y = np.zeros( t.shape, dtype=complex )
    
    # up ramp
    y[0:samplesPerUpRamp] = np.exp( -1j * 2  * np.pi * f0 * tau ) \
    * np.exp( 1j * np.pi * alpha * tau**2 ) \
    * np.exp(  -1j * 2 * np.pi * alpha * (t[0:samplesPerUpRamp]) * tau )
    
    # down ramp
    y[samplesPerUpRamp:2*samplesPerUpRamp] = np.exp( -1j * 2  * np.pi * f0 * tau ) \
    * np.exp( 1j * np.pi * (-alpha) * tau**2 ) \
    * np.exp(  -1j * 2 * np.pi * (-alpha) * (t[samplesPerUpRamp:2*samplesPerUpRamp]) * tau )
    
    return y

In [57]:
def srn( t, r, n ):
    
    tau = 2 * r / c
    
    y = np.zeros( t.shape, dtype=complex )
    
    # up ramp
    y[0:samplesPerUpRamp] = np.exp( -1j * 2  * np.pi * f0 * tau ) \
    * np.exp( 1j * np.pi * alpha * tau**2 ) \
    * np.exp(  -1j * 2 * np.pi * alpha * (t[0:samplesPerUpRamp] + n*T + tau) * tau )
    
    # down ramp
    y[samplesPerUpRamp:2*samplesPerUpRamp] = np.exp( -1j * 2  * np.pi * f0 * tau ) \
    * np.exp( 1j * np.pi * (-alpha) * tau**2 ) \
    * np.exp(  -1j * 2 * np.pi * (-alpha) * (t[samplesPerUpRamp:2*samplesPerUpRamp] + n*T + tau) * tau )
    
    return y

Let us build the equivalent of 10 files of data

In [171]:
a = 0.603354564402
b = a * 180 / (30 * np.pi) # see the Newton method at the bottom of the notebook
def wa( az, rg0 ):
    Rn = ( rg0**2 + az**2 )**0.5
    wa = np.sinc( b * np.arccos( rg0 / Rn ) )**2
    return wa

In [187]:
V = 40
hPlane = 90
hCorner = 90
gr_corner = 100
rg_corner = ( (hPlane - hCorner)**2 + gr_corner**2 )**0.5
az_corner = 180
noise = 1
print("rg_corner = {:.2f}, gr_corner = {:.2f}".format(rg_corner, gr_corner))

rg_corner = 100.00, gr_corner = 100.00


In [188]:
Laz = nbFiles * rampsPerFile * T * V
phi = np.arctan( (Laz/2) / rg_corner )
print( "Laz = {:.0f}, phi = {:.1f}°".format( Laz, phi * 180 / np.pi ) )

Laz = 360, phi = 60.9°


In [189]:
adc = np.zeros( (nbFiles * rampsPerFile, samplesPerRamp), dtype=complex )
for ramp in range( nbFiles * rampsPerFile):
    azn = ramp * T * V
    Rn = ( rg_corner**2 + (azn - az_corner)**2 )**0.5
    adc[ ramp,  0 : samplesPerRamp ] = sb2( t1, Rn ) * wa( azn - az_corner, rg_corner ) \
    + np.random.randn( 6000 ) * noise / (Rn**2)
    if ramp%1000 == 0:
        phi = np.arccos( rg_corner / Rn ) * 180 / np.pi
        print( "{}, wa = {:.1f}, phi = {:.1f}".format( ramp, wa( azn - az_corner, rg_corner ), phi ) )

0, wa = 0.0, phi = 60.9
1000, wa = 0.0, phi = 57.3
2000, wa = 0.0, phi = 52.9
3000, wa = 0.0, phi = 47.2
4000, wa = 0.1, phi = 40.0
5000, wa = 0.2, phi = 31.0
6000, wa = 0.6, phi = 19.8
7000, wa = 0.9, phi = 6.8
8000, wa = 0.9, phi = 6.8
9000, wa = 0.6, phi = 19.8
10000, wa = 0.2, phi = 31.0
11000, wa = 0.1, phi = 40.0
12000, wa = 0.0, phi = 47.2
13000, wa = 0.0, phi = 52.9
14000, wa = 0.0, phi = 57.3


# Save simulated data

In [190]:
firstFile = 0
lastFile = nbFiles - 1
data_dir = "/home/pleroy/DATA/SIMU"

In [191]:
np.tan(45 * np.pi / 180 )

0.99999999999999989

In [192]:
np.save(data_dir + '/adc_files_{}_{}'.format(firstFile, lastFile), adc[:, 0:3000])

In [118]:
adc_reshaped = adc.reshape( (nbFiles * rampsPerFile, samplesPerRamp) )

In [60]:
RD = build_rd_from_data_sim( np.real( adc_reshaped ), samplesPerRamp )
print( "RD.shape = {}".format( RD.shape ) )

shape of the samples matrix = (15000, 6000)
Data are NOT shifted
RD.shape = (15000, 1500)


In [ ]:
np.save(data_dir + '/RD_files_{}_{}'.format(firstFile, lastFile), RD)

In [ ]:
coupling_RD = np.average(RD, 0)
np.save( data_dir + "/coupling_RD_files_{}_{}".format(firstFile, lastFile), coupling_RD )

In [ ]:
samplesPerRamp

In [ ]:
ifft_RD = np.fft.ifft( RD, axis=1 )

In [ ]:
plt.figure()
plt.imshow( 20 * np.log10( np.abs( ifft_RD[::10,:] ) ) )
plt.colorbar()

In [158]:
ifft_adc = np.fft.ifft( adc[:,0:3000], axis=1 )
print( "adc_reshaped.shape = {}, ifft_adc.shape = {}".format( adc_reshaped.shape, ifft_adc.shape ) )

adc_reshaped.shape = (15000, 6000), ifft_adc.shape = (15000, 3000)


In [159]:
plt.figure()
plt.imshow( 20 * np.log10( np.abs( ifft_adc[::10,:] ) ) )
plt.colorbar()

In [161]:
plt.figure()
plt.plot( 20 * np.log10( np.abs( ifft_adc[:,58] ) ), label="58" )
plt.plot( 20 * np.log10( np.abs( ifft_adc[:,59] ) ), label="59" )
plt.plot( 20 * np.log10( np.abs( ifft_adc[:,50] ) ), label="50" )
plt.plot( 20 * np.log10( np.abs( ifft_adc[:,51] ) ), label="51" )
plt.plot( 20 * np.log10( np.abs( ifft_adc[:,52] ) ), label="52" )
plt.plot( 20 * np.log10( np.abs( ifft_adc[:,53] ) ), label="53" )
plt.legend()

In [ ]:
az_corner / V / T

In [ ]:
plt.figure()
plt.plot( 20 * np.log10( np.abs( ifft_RD[7750,:] ) ) )
plt.plot( 20 * np.log10( np.abs( ifft_RD[25000,:] ) ) )

In [ ]:
plt.figure()
plt.plot( 20 * np.log10( np.abs( ifft_RD[::10,500] ) ))
plt.plot( 20 * np.log10( np.abs( ifft_RD[::10,501] ) ))

In [ ]:
RD[100, 0:3000].size

In [ ]:
RD.shape

### Newton's method
f(x) = sin(x) / x

f'(x) = cos(x) / x - sin(x) / x^2

x(n+1) = x(n) - f(x(n)) / f'(x(n))

x(n+1) = x(n) - [f(x(n)) - y0] / f'(x(n))

In [167]:
y0 = 0.1
def newton( xn ):
    f_xn = np.sinc( xn )
    #f_prime_xn = np.cos(xn) / xn - np.sin(xn) / xn**2
    f_prime_xn = np.cos(np.pi * xn) / xn - np.sin(np.pi*xn) / (np.pi*xn**2)
    xn_plus_1 = xn - ( f_xn - y0 ) / f_prime_xn
    return xn_plus_1

In [168]:
x = 1
for k in range( 10 ):
    x = newton( x )
    print( x )

0.9
0.907887181419
0.907928622615
0.90792862378
0.90792862378
0.90792862378
0.90792862378
0.90792862378
0.90792862378
0.90792862378


In [169]:
a = 0.603354564402 # y0 = 0.5
a = 0.90792862378 # y0 = 0.1
b = a * 180 / (30 * np.pi)

In [170]:
x = 30 * np.pi / 180
np.sinc( b * x )

0.099999999999611358

### Analytic signal

In [ ]:
teta = np.arange( -10*np.pi, 10*np.pi, np.pi/10 )

In [ ]:
def as3( y ):
    fft_y = np.fft.fftshift( np.fft.fft( y ) )
    nbPoints = y.shape[0]
    nbPoints2 = int( nbPoints / 2 )
    fft_as_y = np.zeros( (nbPoints,), dtype=complex )
    fft_as_y[0] = fft_y[0]
    fft_as_y[ 1 : nbPoints2 ] = 2 * fft_y[ 1 : nbPoints2 ]
    fft_as_y[0] = fft_y[0]
    fft_as_y[nbPoints2] = fft_y[nbPoints2]
    return np.fft.ifft( np.fft.ifftshift( fft_as_y ) ) / 2

In [ ]:
def as2( y ):
    return signal.hilbert( y )/2

In [ ]:
def RD_realtocomp2( RD ):

    Nf  = RD.size

    # if Nf is a multiple of 2, fftshift(fftshift(X)) = X
    if ( ( Nf % 2 ) != 0 ):
        error( "Nf should be a multiple of 2" )

    h = np.zeros( Nf )
    h[ 0 ] = 1
    h[ int( Nf / 2) ] = 1
    h[ 1 : int(Nf / 2) ] = 2

    #RDs = np.fft.ifft( np.fft.fftshift( RD ) )
    RDs = np.fft.ifft( RD )

    RDh = RDs * h

    #RDc = np.fft.fftshift( np.fft.fft( RDh ) )
    RDc = np.fft.fft( RDh )

    return RDc/2

In [ ]:
phi0 = 30 * 180 / np.pi
y = np.cos(teta - phi0) + np.cos( 2 * teta - phi0)
y = teta
expy = np.exp(1j * (teta-phi0) ) / 2
as_y = RD_realtocomp2( y )
as2_y = as2( y )
as3_y = as3( y )
fft_y = np.fft.fft( y )
fft_expy = np.fft.fft( expy )

plt.figure()

plt.subplot(121)
plt.plot( np.real( np.fft.fft( as3_y ) ), "D-", label="as3_y" )
plt.plot( np.real( np.fft.fft( as2_y ) ), '.-', label="as2_y (hilbert)" )
plt.plot( np.real( np.fft.fft( as_y ) ), "x-", label="as_y (RD_realtocomp)", color="yellow" )
plt.plot( np.real( fft_y ), label="fft_y" )
plt.grid()
plt.legend()

plt.subplot(122)
plt.plot( np.imag( np.fft.fft( as3_y ) ), "D-" )
plt.plot( np.imag( np.fft.fft( as2_y ) ), '.-' )
plt.plot( np.imag( np.fft.fft( as_y ) ), "x-", color="yellow" )
plt.plot( np.imag( fft_y ) )
plt.grid()

In [ ]:
plt.figure()

as_y = RD_realtocomp( y )
as_y_ifftshift = np.fft.ifftshift(RD_realtocomp( y ))

plt.subplot(121)
plt.plot( np.real( np.fft.fft( as_y ) ), "x-", label="real as_y" )
plt.plot( np.real( np.fft.fft( as_y_ifftshift ) ), ".-", label="real as_y_ifftshift" )
plt.grid()
plt.legend()

plt.subplot(122)
plt.plot( np.imag( np.fft.fft( as_y ) ), "x-", label="imag as_y" )
plt.plot( np.imag( np.fft.fft( as_y_ifftshift ) ), ".-", label="imag as_y_ifftshift" )
plt.grid()

In [ ]:
plt.figure()

as_y = RD_realtocomp( y )
as_y_ifftshift = np.fft.ifftshift(RD_realtocomp( y ))

plt.subplot(121)
plt.plot( np.real( as_y ), "x-", label="real as_y" )
plt.plot( np.real( as_y_ifftshift ), ".-", label="real as_y_ifftshift" )
plt.grid()
plt.legend()

plt.subplot(122)
plt.plot( np.imag( as_y ), "x-", label="imag as_y" )
plt.plot( np.imag( as_y_ifftshift ), ".-", label="imag as_y_ifftshift" )
plt.grid()